In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e12/sample_submission.csv
/kaggle/input/playground-series-s4e12/train.csv
/kaggle/input/playground-series-s4e12/test.csv


# Libraries¶

In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder



import warnings
warnings.filterwarnings("ignore")

# GPU

In [3]:
import torch

def check_gpu_status():
    if torch.cuda.is_available():
        print(f"可用GPU数量: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
            print(f"显存使用情况: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB 已用, {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB 总分配")
            print(f"显存空余: {torch.cuda.memory_reserved(i) - torch.cuda.memory_allocated(i):.2f} 字节\n")
    else:
        print("未检测到 GPU！")

check_gpu_status()

可用GPU数量: 2
GPU 0: Tesla T4
显存使用情况: 0.00 MB 已用, 0.00 MB 总分配
显存空余: 0.00 字节

GPU 1: Tesla T4
显存使用情况: 0.00 MB 已用, 0.00 MB 总分配
显存空余: 0.00 字节



# IMPORT DATA

In [4]:
train = pd.read_csv("/kaggle/input/playground-series-s4e12/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s4e12/test.csv")

sample = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True) 

# DATA Processing¶

In [5]:
def date(Df):

    Df['Policy Start Date'] = pd.to_datetime(Df['Policy Start Date'])
    Df['Year'] = Df['Policy Start Date'].dt.year
    Df['Day'] = Df['Policy Start Date'].dt.day
    Df['Month'] = Df['Policy Start Date'].dt.month
    Df['Month_name'] = Df['Policy Start Date'].dt.month_name()
    Df['Day_of_week'] = Df['Policy Start Date'].dt.day_name()
    Df['Week'] = Df['Policy Start Date'].dt.isocalendar().week
    Df['Year_sin'] = np.sin(2 * np.pi * Df['Year'])
    Df['Year_cos'] = np.cos(2 * np.pi * Df['Year'])
    Df['Month_sin'] = np.sin(2 * np.pi * Df['Month'] / 12) 
    Df['Month_cos'] = np.cos(2 * np.pi * Df['Month'] / 12)
    Df['Day_sin'] = np.sin(2 * np.pi * Df['Day'] / 31)  
    Df['Day_cos'] = np.cos(2 * np.pi * Df['Day'] / 31)
    Df['Group']=(Df['Year']-2020)*48+Df['Month']*4+Df['Day']//7
    
    Df.drop('Policy Start Date', axis=1, inplace=True)

    return Df

In [6]:
train = date(train)
test = date(test)

cat_cols = [col for col in train.columns if train[col].dtype == 'object']
feature_cols = list(test.columns)

In [7]:
class CategoricalEncoder:
    def __init__(self, train, test):
        self.train = train
        self.test = test

    def frequency_encode(self, cat_cols, feature_cols, drop_org=False):
        combined = pd.concat([self.train, self.test], axis=0, ignore_index=True)

        new_cat_cols = [] 
        for col in cat_cols:
            freq_encoding = combined[col].value_counts().to_dict()
            
            self.train[f"{col}_freq"] = self.train[col].map(freq_encoding).astype('float')
            self.test[f"{col}_freq"] = self.test[col].map(freq_encoding).astype('float')

            new_col_name = f"{col}_freq"
            new_cat_cols.append(new_col_name)
            feature_cols.append(new_col_name)
            if drop_org:
                feature_cols.remove(col)

        return self.train, self.test, new_cat_cols, feature_cols

In [8]:
encoder = CategoricalEncoder(train, test)
train, test, cat_cols, feature_cols = encoder.frequency_encode(cat_cols, feature_cols, drop_org=True)

train = train[feature_cols + ['Premium Amount']]
test = test[feature_cols]


In [9]:
train.head()

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Year,Day,...,Occupation_freq,Location_freq,Policy Type_freq,Customer Feedback_freq,Smoking Status_freq,Exercise Frequency_freq,Property Type_freq,Month_name_freq,Day_of_week_freq,Premium Amount
0,19.0,10049.0,1.0,22.598761,2.0,17.0,372.0,5.0,2023,23,...,470636.0,663201.0,669475.0,625952.0,996268.0,510693.0,667500.0,162307.0,284861.0,2869.0
1,39.0,31678.0,3.0,15.569731,1.0,12.0,694.0,2.0,2023,12,...,NaN,668067.0,665822.0,629122.0,1003732.0,498230.0,667500.0,164442.0,287191.0,1483.0
2,23.0,25602.0,3.0,47.177549,1.0,14.0,NaN,3.0,2023,30,...,470636.0,668732.0,669475.0,614826.0,1003732.0,510693.0,667500.0,165556.0,284861.0,567.0
3,21.0,141855.0,2.0,10.938144,1.0,0.0,367.0,1.0,2024,12,...,NaN,668067.0,664703.0,625952.0,1003732.0,491143.0,666022.0,164442.0,287424.0,765.0
4,21.0,39651.0,1.0,20.376094,0.0,8.0,598.0,4.0,2021,1,...,470636.0,668067.0,669475.0,625952.0,1003732.0,510693.0,667500.0,162307.0,287424.0,2022.0


In [10]:
X = train.drop('Premium Amount', axis=1)  
y = train['Premium Amount']

y_log = np.log1p(y)

In [11]:
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [12]:
def train_model():
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    oof = np.zeros(len(X))
    models = []

    for fold, (train_idx, valid_idx) in enumerate(kf.split(X)):
        print(f"Fold {fold + 1}")
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y_log.iloc[train_idx], y_log.iloc[valid_idx]

        model = CatBoostRegressor(
            iterations=3000,
            learning_rate=0.05,
            depth=6,
            eval_metric="RMSE",
            random_seed=42,
            verbose=200,
            task_type='GPU',
            l2_leaf_reg =  0.7,
        )
        
        model.fit(X_train,
                  y_train,
                  eval_set=(X_valid, y_valid), 
                  early_stopping_rounds=300,
                  # cat_features=cat_cols,
                 )
        models.append(model)
        oof[valid_idx] = np.maximum(0, model.predict(X_valid))
        fold_rmsle = rmsle(np.expm1(y_valid), np.expm1(oof[valid_idx]))
        print(f"Fold {fold + 1} RMSLE: {fold_rmsle}")
        
    return models, oof
models,oof = train_model()

Fold 1
0:	learn: 1.0927138	test: 1.0934793	best: 1.0934793 (0)	total: 346ms	remaining: 17m 18s
200:	learn: 1.0488506	test: 1.0501858	best: 1.0501858 (200)	total: 1.97s	remaining: 27.4s
400:	learn: 1.0470857	test: 1.0491505	best: 1.0491505 (400)	total: 3.67s	remaining: 23.8s
600:	learn: 1.0459035	test: 1.0488293	best: 1.0488293 (598)	total: 5.41s	remaining: 21.6s
800:	learn: 1.0448941	test: 1.0486400	best: 1.0486360 (794)	total: 7.14s	remaining: 19.6s
1000:	learn: 1.0439487	test: 1.0485827	best: 1.0485793 (987)	total: 8.87s	remaining: 17.7s
1200:	learn: 1.0430424	test: 1.0485649	best: 1.0485592 (1166)	total: 10.6s	remaining: 15.9s
1400:	learn: 1.0421672	test: 1.0485818	best: 1.0485518 (1357)	total: 12.3s	remaining: 14s
1600:	learn: 1.0413562	test: 1.0486210	best: 1.0485518 (1357)	total: 14.1s	remaining: 12.3s
bestTest = 1.048551829
bestIteration = 1357
Shrink model to first 1358 iterations.
Fold 1 RMSLE: 1.0485518142307853
Fold 2
0:	learn: 1.0929991	test: 1.0922494	best: 1.0922494 (0)	t

In [13]:
print(rmsle(y, np.expm1(oof)))

1.0479970099105032


In [14]:
test_predictions = np.zeros(len(test))

for model in models:
    test_predictions += np.maximum(0, np.expm1(model.predict(test))) / len(models)


sample['Premium Amount'] = test_predictions
sample.to_csv('submission.csv', index = False)
sample.head()

,id,Premium Amount
0,1200000,786.039858
1,1200001,787.390070
2,1200002,778.321892
3,1200003,793.235726
4,1200004,749.701754


In [15]:
from sklearn.model_selection import ParameterSampler
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from tqdm import tqdm

# 定义超参数搜索空间
param_grid = {
    'iterations': [2000, 3000, 4000],
    'learning_rate': [0.01, 0.03, 0.05],
    'depth': [4, 6, 8, 10],
    'l2_leaf_reg': [0.5, 0.7, 1.0, 2.0],
    'bagging_temperature': [0.5, 1.0, 1.5],
    'random_strength': [1, 2, 5]
}

# 随机抽取参数
n_searches = 15
param_list = list(ParameterSampler(param_grid, n_iter=n_searches, random_state=42))

def random_search_train(X, y, param_list, n_splits=5, random_state=42):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    best_score = float('inf')
    best_params = None
    best_models = None
    results = []

    for idx, params in enumerate(param_list):
        print(f"Search {idx+1}/{len(param_list)} with params: {params}")
        oof = np.zeros(len(X))
        models = []
        fold_scores = []

        for fold, (train_idx, valid_idx) in enumerate(kf.split(X)):
            print(f"  Fold {fold + 1}")
            X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
            y_train, y_valid = np.log1p(y.iloc[train_idx]), np.log1p(y.iloc[valid_idx])

            model = CatBoostRegressor(
                **params,
                random_seed=random_state,
                eval_metric="RMSE",
                verbose=0,
                task_type='GPU',
            )
            
            model.fit(
                X_train, y_train,
                eval_set=(X_valid, y_valid),
                early_stopping_rounds=300,
            )
            models.append(model)

            # 预测和评估
            oof[valid_idx] = np.maximum(0, model.predict(X_valid))
            fold_rmsle = rmsle(np.expm1(y_valid), np.expm1(oof[valid_idx]))
            fold_scores.append(fold_rmsle)

            print(f"    Fold {fold + 1} RMSLE: {fold_rmsle:.4f}")
        
        mean_score = np.mean(fold_scores)
        print(f"Mean RMSLE for Search {idx+1}: {mean_score:.4f}")

        # 保存最佳模型
        if mean_score < best_score:
            best_score = mean_score
            best_params = params
            best_models = models

        # 记录结果
        results.append({'params': params, 'mean_rmsle': mean_score})
    
    results_df = pd.DataFrame(results)
    print("\nBest Parameters:", best_params)
    print(f"Best RMSLE: {best_score:.4f}")
    return best_models, best_params, results_df

# 执行随机搜索训练
models, best_params, results_df = random_search_train(X, y, param_list)


Search 1/15 with params: {'random_strength': 2, 'learning_rate': 0.03, 'l2_leaf_reg': 1.0, 'iterations': 3000, 'depth': 6, 'bagging_temperature': 1.0}
  Fold 1
    Fold 1 RMSLE: 1.0486
  Fold 2
    Fold 2 RMSLE: 1.0478
  Fold 3
    Fold 3 RMSLE: 1.0485
  Fold 4
    Fold 4 RMSLE: 1.0467
  Fold 5
    Fold 5 RMSLE: 1.0482
Mean RMSLE for Search 1: 1.0480
Search 2/15 with params: {'random_strength': 2, 'learning_rate': 0.03, 'l2_leaf_reg': 1.0, 'iterations': 2000, 'depth': 10, 'bagging_temperature': 0.5}
  Fold 1
    Fold 1 RMSLE: 1.0480
  Fold 2
    Fold 2 RMSLE: 1.0471
  Fold 3
    Fold 3 RMSLE: 1.0479
  Fold 4
    Fold 4 RMSLE: 1.0460
  Fold 5
    Fold 5 RMSLE: 1.0476
Mean RMSLE for Search 2: 1.0473
Search 3/15 with params: {'random_strength': 2, 'learning_rate': 0.05, 'l2_leaf_reg': 1.0, 'iterations': 4000, 'depth': 8, 'bagging_temperature': 1.0}
  Fold 1
    Fold 1 RMSLE: 1.0484
  Fold 2
    Fold 2 RMSLE: 1.0475
  Fold 3
    Fold 3 RMSLE: 1.0483
  Fold 4
    Fold 4 RMSLE: 1.0464
  Fold

In [16]:
test_predictions = np.zeros(len(test))

for model in models:
    test_predictions += np.maximum(0, np.expm1(model.predict(test))) / len(models)


sample['Premium Amount'] = test_predictions
sample.to_csv('submission1.csv', index = False)
sample.head()

,id,Premium Amount
0,1200000,815.438101
1,1200001,795.125828
2,1200002,785.394724
3,1200003,797.848012
4,1200004,745.831867


# The end